In [1]:
# required to get access to model_stacking package
import os
os.chdir('../..')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
from framework.model_stacking import getConfigParameters
import pickle

retrieving configuration file: config.yml from current working directory


In [3]:
CONFIG = getConfigParameters()
ROOT_DIR = CONFIG['ROOT_DIR']
DATA_DIR = CONFIG['DATA_DIR']
ID_VAR = CONFIG['ID_VAR']
TARGET_VAR = CONFIG['TARGET_VAR']

In [4]:
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer

## Get Training Data

In [5]:
# get training data
train_df = pd.read_csv(os.path.join(ROOT_DIR,DATA_DIR,'KFS01','train.csv.gz'))

In [6]:
X_train = train_df.drop(ID_VAR + [TARGET_VAR],axis=1)
y_train = train_df.loc[:,TARGET_VAR]

In [7]:
X_train.shape

(4459, 4730)

In [8]:
y_train.shape

(4459,)

In [9]:
y_train[:10]

0    17.453097
1    13.304687
2    16.118096
3    14.508658
4    16.482739
5    14.845130
6    12.007628
7    13.304687
8    13.794288
9    13.038984
Name: target, dtype: float64

## Setup pipeline for hyper-parameter tuning

In [10]:
# set up pipeline
pipe = Pipeline([('nnet',MLPRegressor(max_iter=700))])

In [32]:
def kag_rmsle(y,y_hat):
    return np.sqrt(mean_squared_error(y,y_hat))

this_scorer = make_scorer(kag_rmsle, greater_is_better=False)

In [33]:
param_grid = dict(nnet__hidden_layer_sizes=[(5,1),(5,)])
grid_search = GridSearchCV(pipe, param_grid=param_grid,scoring=this_scorer,cv=5,verbose=2,n_jobs=1,
                          refit=False)

In [34]:
grid_search.fit(X_train,y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] nnet__hidden_layer_sizes=(5, 1) .................................


/Users/jim/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.3min remaining:    0.0s


[CV] .................. nnet__hidden_layer_sizes=(5, 1), total= 1.3min
[CV] nnet__hidden_layer_sizes=(5, 1) .................................


/Users/jim/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


[CV] .................. nnet__hidden_layer_sizes=(5, 1), total= 1.3min
[CV] nnet__hidden_layer_sizes=(5, 1) .................................


...

[CV] .................. nnet__hidden_layer_sizes=(5, 1), total= 1.3min
[CV] nnet__hidden_layer_sizes=(5,) ...................................
[CV] .................... nnet__hidden_layer_sizes=(5,), total=  38.6s
[CV] nnet__hidden_layer_sizes=(5,) ...................................
[CV] .................... nnet__hidden_layer_sizes=(5,), total=  57.8s
[CV] nnet__hidden_layer_sizes=(5,) ...................................
[CV] .................... nnet__hidden_layer_sizes=(5,), total=  53.5s
[CV] nnet__hidden_layer_sizes=(5,) ...................................


/Users/jim/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


[CV] .................... nnet__hidden_layer_sizes=(5,), total= 1.4min
[CV] nnet__hidden_layer_sizes=(5,) ...................................
[CV] .................... nnet__hidden_layer_sizes=(5,), total=  33.6s


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 10.2min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('nnet', MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=700, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'nnet__hidden_layer_sizes': [(5, 1), (5,)]},
       pre_dispatch='2*n_jobs', refit=False, return_train_score=True,
       scoring=make_scorer(kag_rmsle, greater_is_better=False), verbose=2)

In [35]:
grid_search.best_params_

{'nnet__hidden_layer_sizes': (5, 1)}

In [36]:
grid_search.best_score_

-3.6783800706480596

In [37]:
grid_search.cv_results_

{'mean_fit_time': array([ 68.58774309,  53.17852821]),
 'mean_score_time': array([ 0.00542865,  0.00500793]),
 'mean_test_score': array([-3.67838007, -3.75431456]),
 'mean_train_score': array([-3.20334559, -1.58525507]),
 'param_nnet__hidden_layer_sizes': masked_array(data = [(5, 1) (5,)],
              mask = [False False],
        fill_value = ?),
 'params': ({'nnet__hidden_layer_sizes': (5, 1)},
  {'nnet__hidden_layer_sizes': (5,)}),
 'rank_test_score': array([1, 2], dtype=int32),
 'split0_test_score': array([-4.22215631, -3.34518213]),
 'split0_train_score': array([-4.11892293, -0.92202645]),
 'split1_test_score': array([-4.10775146, -4.02555327]),
 'split1_train_score': array([-4.14868659, -0.85746633]),
 'split2_test_score': array([-2.82575763, -3.92754302]),
 'split2_train_score': array([-2.82218153, -0.87931378]),
 'split3_test_score': array([-3.26471061, -4.2944781 ]),
 'split3_train_score': array([-0.91604212, -4.33098164]),
 'split4_test_score': array([-3.97185335, -3.178170

In [38]:
grid_search.n_splits_

5

In [39]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_nnet__hidden_layer_sizes,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,68.587743,0.005429,-3.678380,-3.203346,"(5, 1)","{'nnet__hidden_layer_sizes': (5, 1)}",1,-4.222156,-4.118923,-4.107751,...,-2.825758,-2.822182,-3.264711,-0.916042,-3.971853,-4.010895,17.965865,0.001781,0.541202,1.245884
1,53.178528,0.005008,-3.754315,-1.585255,"(5,)","{'nnet__hidden_layer_sizes': (5,)}",2,-3.345182,-0.922026,-4.025553,...,-3.927543,-0.879314,-4.294478,-4.330982,-3.178170,-0.936487,17.216846,0.000470,0.422969,1.373158


In [40]:
np.sqrt(13)

3.6055512754639891